# Xây dựng mô hình từ giải thuật SVM trên dữ liệu các con thú trong rừng.

#### nạp dữ liệu

In [3]:
import pandas as pd
import numpy as np

# 1. Nạp dữ liệu
# Đảm bảo bạn thay 'zoo.csv' bằng tên file bạn đã tải về
df = pd.read_csv('datasets/zoo.csv')

In [4]:
df.head()

,animal_name,hair,feathers,eggs,milk,airborne,aquatic,predator,toothed,backbone,breathes,venomous,fins,legs,tail,domestic,catsize,class_type
0,aardvark,1,0,0,1,0,0,1,1,1,1,0,0,4,0,0,1,1
1,antelope,1,0,0,1,0,0,0,1,1,1,0,0,4,1,0,1,1
2,bass,0,0,1,0,0,1,1,1,1,0,0,1,0,1,0,0,4
3,bear,1,0,0,1,0,0,1,1,1,1,0,0,4,0,0,1,1
4,boar,1,0,0,1,0,0,1,1,1,1,0,0,4,1,0,1,1


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101 entries, 0 to 100
Data columns (total 18 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   animal_name  101 non-null    object
 1   hair         101 non-null    int64 
 2   feathers     101 non-null    int64 
 3   eggs         101 non-null    int64 
 4   milk         101 non-null    int64 
 5   airborne     101 non-null    int64 
 6   aquatic      101 non-null    int64 
 7   predator     101 non-null    int64 
 8   toothed      101 non-null    int64 
 9   backbone     101 non-null    int64 
 10  breathes     101 non-null    int64 
 11  venomous     101 non-null    int64 
 12  fins         101 non-null    int64 
 13  legs         101 non-null    int64 
 14  tail         101 non-null    int64 
 15  domestic     101 non-null    int64 
 16  catsize      101 non-null    int64 
 17  class_type   101 non-null    int64 
dtypes: int64(17), object(1)
memory usage: 14.3+ KB


#### Tiền xử lý

In [9]:
# 1. Xác định cột mục tiêu (target)
target = 'class_type'

# 2. Tạo X (features)
# Chúng ta sẽ LOẠI BỎ cột target ('class_type') 
# và cột chữ vô dụng ('animal_name')
X = df.drop(columns=[target, 'animal_name'])

# 3. Tạo y (target)
y = df[target]

#### Chuẩn hóa dữ liệu

In [10]:
from sklearn.preprocessing import StandardScaler

# Bây giờ X của bạn đã toàn là số (gồm số gốc + số mới từ get_dummies)
scaler = StandardScaler()

# Fit và transform toàn bộ X
X_scaled = scaler.fit_transform(X)

#### Chia dữ liệu

In [11]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, 
    test_size=0.2, 
    random_state=42 # Chọn 1 số bất kỳ
)

#### Xây dựng và tối ưu SVM

In [12]:
from sklearn import svm
from sklearn.model_selection import GridSearchCV

svc_model = svm.SVC()
param_grid = {
    'C': [0.1, 1, 10, 100],
    'kernel': ['linear', 'rbf']
}

grid = GridSearchCV(svc_model, param_grid, refit=True, verbose=2, cv=4, scoring='accuracy')
grid.fit(X_train, y_train)

# In ra mô hình tốt nhất
print(grid.best_params_)

Fitting 4 folds for each of 8 candidates, totalling 32 fits
[CV] END ...............................C=0.1, kernel=linear; total time=   0.0s
[CV] END ...............................C=0.1, kernel=linear; total time=   0.0s
[CV] END ...............................C=0.1, kernel=linear; total time=   0.0s
[CV] END ...............................C=0.1, kernel=linear; total time=   0.0s
[CV] END ..................................C=0.1, kernel=rbf; total time=   0.0s
[CV] END ..................................C=0.1, kernel=rbf; total time=   0.0s
[CV] END ..................................C=0.1, kernel=rbf; total time=   0.0s
[CV] END ..................................C=0.1, kernel=rbf; total time=   0.0s
[CV] END .................................C=1, kernel=linear; total time=   0.0s
[CV] END .................................C=1, kernel=linear; total time=   0.0s
[CV] END .................................C=1, kernel=linear; total time=   0.0s
[CV] END .................................C=1, ke

#### Đánh giá mô hình

In [13]:
from sklearn.metrics import classification_report, accuracy_score

best_svm = grid.best_estimator_
predictions = best_svm.predict(X_test)

print(f"Độ chính xác (Accuracy): {accuracy_score(y_test, predictions):.4f}")
print(classification_report(y_test, predictions))

Độ chính xác (Accuracy): 0.9524
              precision    recall  f1-score   support

           1       1.00      1.00      1.00        12
           2       1.00      1.00      1.00         2
           3       0.00      0.00      0.00         1
           4       0.67      1.00      0.80         2
           6       1.00      1.00      1.00         3
           7       1.00      1.00      1.00         1

    accuracy                           0.95        21
   macro avg       0.78      0.83      0.80        21
weighted avg       0.92      0.95      0.93        21



c:\Users\ADMIN\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\ADMIN\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\ADMIN\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


Kết thúc